# The (linear) shallow water equations

Here we'll look at a linearization of the shallow water equations.
This is a 2nd-order in time problem.
To solve it, we'll introduce an additional variable to reduce it to first-order in time.
This will force us to learn how to solve multi-component systems.
We'll also try out the irksome implementation of the Gauss timestepping scheme, i.e. collocation based on the Gauss quadrature points.

In [ ]:
import firedrake
from firedrake import Constant, inner, grad, dx
import irksome
from irksome import Dt
import matplotlib.pyplot as plt
import numpy as np

As a spatial domain, we'll consider a circular lake 100m wide.
There's a built-in function to create a mesh of the unit disk.
We'll make it 100m wide in a hacky way by altering the raw data that lives inside the mesh coordinates.

In [ ]:
mesh = firedrake.UnitDiskMesh(4)
radius = 100.0
mesh.coordinates.dat.data[:] *= radius

In [ ]:
element = firedrake.FiniteElement("CG", "triangle", 1)
Q = firedrake.FunctionSpace(mesh, element)

The variational form of the wave equation we're working with here is
$$\int_\Omega\left(\partial_t^2h\cdot\phi + gH\nabla h\cdot\nabla\phi\right)dx = 0$$
for all test functions $\phi$.
Note that we now have a second time derivative of $h$, so we'll want to introduce a new variable
$$u = \partial_t h$$
to reduce the system to first order.
What's the variational form now that we've added a new variable?
We now have not just one test function but two:
$$\int_\Omega\left\{\partial_tu\cdot\phi + gH\nabla h\cdot\nabla\phi + (\partial_th - u)v\right\}dx = 0$$
for all pairs of test functions $\phi$, $v$.

First, we'll make the input data.
We'll start with a flat bottom ($H$ = 5m).
I'll make you do this again with variable bottom topography for homework.

In [ ]:
g = Constant(9.8)
H = Constant(5.0)

The wave speed for shallow water waves is $c = \sqrt{gH}$.
(Check the units!)
The radius of the domain is 100m, so we can calculate a timescale for a wave to cross the whole domain is
$$T = R / \sqrt{gH}.$$
We can pick a sensible timestep in the same way -- how long does the wave take to cross a single grid cell?
$$\delta t = \delta x / \sqrt{gH}$$

In [ ]:
wave_speed = np.sqrt(float(g) * float(H))

final_time = 4 * radius / wave_speed
cell_diam = mesh.cell_sizes.dat.data_ro.max() / 2
timestep = cell_diam / wave_speed / 2
num_steps = int(final_time / timestep)
print(final_time, timestep, num_steps)

If we want to solve simultaneously for both $\phi$ and $u$, we'll need to make a new function space containing *pairs* of variables.

In [ ]:
Z = Q * Q

We can now see that the function space $Z$ has a new type, a *mixed* finite element:

In [ ]:
Z.ufl_element()

We can access the components of a function in a mixed space using the `.subs` method.
Here, I'm creating a function $z$ in the space $Z$, and assigning a small perturbation in the wave height to the 0th component.

In [ ]:
z = firedrake.Function(Z)

x = firedrake.SpatialCoordinate(mesh)
x_0 = Constant((radius / 4, radius / 4))
r = Constant(radius / 8)
δh = Constant(H / 20)
expr = δh * firedrake.exp(-inner(x - x_0, x - x_0) / r**2)

z.sub(0).interpolate(expr);

And in order to make sure we got something sensible looking, I've plotted the result below.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(z.sub(0), axes=ax)
fig.colorbar(colors);

In order to get some symbols representing $h$ and $u$ separately, we have to first create a function that lives in $Z$ and then split it in two.
The function `firedrake.split` takes a function in a mixed space, and it returns us a tuple of symbolic objects that represent its component functions.
Here I'm adopting the convention that the thickness variable comes first and the surface height change second.

In [ ]:
h, u = firedrake.split(z)

w = firedrake.TestFunction(Z)
ϕ, v = firedrake.split(w)

F = (Dt(u) * ϕ + g * H * inner(grad(h), grad(ϕ)) + (Dt(h) - u) * v) * dx

Now we'll do the whole song and dance with Irksome.
I've picked the Gauss-Legendre method because it works better for wave-type problems.

In [ ]:
t = firedrake.Constant(0.0)
dt = firedrake.Constant(timestep)
method = irksome.GaussLegendre(1)
solver = irksome.TimeStepper(F, method, t, dt, z)

You'll notice that in the middle of the loop I'm calling a method `z.subfunctions`.
The function `firedrake.split` gives us *symbolic* answers.
The method `.subfunctions` gives us *numeric* answers.
The last bit of the loop pulls out the thickness and height change fields and packs them into a list so that we can make a movie at the end.

In [ ]:
hs = [z.sub(0).copy(deepcopy=True)]
us = [z.sub(1).copy(deepcopy=True)]
for step in range(num_steps):
    t.assign(t + dt)
    solver.advance()

    h, u = z.subfunctions
    hs.append(h.copy(deepcopy=True))
    us.append(u.copy(deepcopy=True))

Ooh pretty

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(hs[0], vmin=-float(δh), vmax=+float(δh), num_sample_points=4, axes=ax)
fig.colorbar(colors)

fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(h):
    colors.set_array(fn_plotter(h))

animation = FuncAnimation(fig, animate, hs, interval=1e3/12)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

Let's try to see how well this method conserved energy.

In [ ]:
energies = np.array(
    [
        firedrake.assemble((u**2 + g * H * inner(grad(h), grad(h))) * dx)
        for h, u in zip(hs, us)
    ]
)

<a href="https://imgflip.com/i/8opj3r"><img src="https://i.imgflip.com/8opj3r.jpg" title="made at imgflip.com"/></a><div><a href="https://imgflip.com/memegenerator">from Imgflip Meme Generator</a></div>

In [ ]:
print(f"Average energy: {energies.mean():.1f}")
print(f"Energy drift:   {energies.max() - energies.min():.1g}")